In [6]:
import pandas as pd
import numpy as np
from scipy import stats
import ast
import networkx as nx
from pyvis.network import Network
import graphviz
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
from torch_geometric.explain.metric import groundtruth_metrics
import torch
import numba
import seaborn as sns
import iqplot
from sklearn import metrics
import bokeh.models
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
ec1 = '#1A85FF'
ec0_0 = '#D41159'
ec0_1 = '#ED9FBC'

In [3]:
basebetaall = pd.read_csv('data/xAI Comparison/baseBetaExpxAIProbsVSGT.csv')
basebetaidx = basebetaall[basebetaall['Idx'] == 11214]
basebetagraph = basebetaidx[basebetaidx['Run'] == 5]
p_edges = basebetagraph[basebetagraph['GT'] == 1]
neg_edges = basebetagraph[basebetagraph['GT'] == 1]
fn_edges = neg_edges[neg_edges['Beta Probability'] < 0.5]
p_set = set()
p1s = list(p_edges['P1'])
p2s = list(p_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    p_set.add((p1, p2))
fn_set = set()
width = dict()
wtr = set()
p1s = list(fn_edges['P1'])
p2s = list(fn_edges['P2'])
prob = list(fn_edges['Beta Probability'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    fn_set.add((p1, p2))
    if (p1, p2) not in wtr:
        if p1 in width.keys():
            width[p1][p2] = prob[i]
        else:
            width[p1] = {p2: prob[i]}
g = Network(notebook=True, directed=True)
ei1 = list(basebetagraph['P1'])
ei2 = list(basebetagraph['P2'])
prob = list(basebetagraph['Beta Probability'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
if len(fn_set) > 0:
    for (p1, p2) in fn_set:
        b1.append(p1)
        b2.append(p2)
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in p_set:
        if (p1, p2) in fn_set:
            edge['color'] = ec0_1
        else:
            edge["color"] = ec1
    else:
        edge["color"] = ec0_0
g.show('data/Vis/BaselineGraphBeta.html')

data/Vis/BaselineGraphBeta.html


In [4]:
basebetaall = pd.read_csv('data/xAI Comparison/baseBetaExpxAIProbsVSGT.csv')
basebetaidx = basebetaall[basebetaall['Idx'] == 11214]
basebetagraph = basebetaidx[basebetaidx['Run'] == 5]
p_edges = basebetagraph[basebetagraph['GT'] == 1]
neg_edges = basebetagraph[basebetagraph['GT'] == 1]
fn_edges = neg_edges[neg_edges['Beta Probability'] < 0.5]
p_set = set()
p1s = list(p_edges['P1'])
p2s = list(p_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    p_set.add((p1, p2))
fn_set = set()
width = dict()
wtr = set()
p1s = list(fn_edges['P1'])
p2s = list(fn_edges['P2'])
prob = list(fn_edges['Beta Probability'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    fn_set.add((p1, p2))
    if (p1, p2) not in wtr:
        if p1 in width.keys():
            width[p1][p2] = prob[i]
        else:
            width[p1] = {p2: prob[i]}
g = Network(notebook=True, directed=True)
ei1 = list(basebetagraph['P1'])
ei2 = list(basebetagraph['P2'])
prob = list(basebetagraph['Beta Probability'])
b1 = []
b2 = []
n = []
c = []
pr = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        pr.append(prob[i])
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
if len(fn_set) > 0:
    for (p1, p2) in fn_set:
        b1.append(p1)
        b2.append(p2)
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5 * (width[b1[i]][b2[i]] - np.mean(pr)) / np.std(pr)
    g.add_edge(b1[i], b2[i], width=w)
col = []
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    w = (width[b1[i]][b2[i]] - np.mean(pr)) / np.std(pr)
    edge['color'] = lighten('#000000', w)
g.show('data/Vis/BaselineGraphBetaProbs.html')

NameError: name 'lighten' is not defined

In [ ]:
np.unique(col)

In [ ]:
basegnnall = pd.read_csv('data/xAI Comparison/baseGNNExpxAIProbsVSGT.csv')
basegnnidx = basegnnall[basegnnall['Idx'] == 11214]
br = 0
bestacc = 0
bestf1 = 0
runs = np.unique(basegnnidx['Run'])
for run in runs:
    curr = basegnnidx[basegnnidx['Run'] == run]
    probs = torch.tensor(list(curr['GNN Probability']), dtype=torch.float32)
    gt = torch.tensor(list(curr['GT']), dtype=torch.int64)
    met = groundtruth_metrics(probs, gt)
    acc = met[0]
    f1 = met[3]
    if acc >= bestacc and f1 >= bestf1:
        br = run
        bestacc = acc
        bestf1 = f1
print(bestacc, bestf1, br)

In [ ]:
basegnngraph = basegnnidx[basegnnidx['Run'] == br]
p_edges = basegnngraph[basegnngraph['GT'] == 1]
neg_edges = basegnngraph[basegnngraph['GT'] == 1]
fn_edges = neg_edges[neg_edges['GNN Probability'] < 0.5]
p_set = set()
p1s = list(p_edges['P1'])
p2s = list(p_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    p_set.add((p1, p2))
fn_set = set()
p1s = list(fn_edges['P1'])
p2s = list(fn_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    fn_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(basegnngraph['P1'])
ei2 = list(basegnngraph['P2'])
prob = list(basegnngraph['GNN Probability'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
if len(fn_set) > 0:
    for (p1, p2) in fn_set:
        b1.append(p1)
        b2.append(p2)
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in p_set:
        if (p1, p2) in fn_set:
            edge['color'] = ec0_1
        else:
            edge["color"] = ec1
    else:
        edge["color"] = ec0_0
g.show('data/Vis/BaselineGraphGNN.html')

In [ ]:
basegnnall = pd.read_csv('data/xAI Comparison/baseGNNExpxAIProbsVSGT.csv')
basegnnidx = basegnnall[basegnnall['Idx'] == 11214]
basegnngraph = basegnnidx[basegnnidx['Run'] == br]
g = Network(notebook=True, directed=True)
ei1 = list(basegnngraph['P1'])
ei2 = list(basegnngraph['P2'])
prob = list(basegnngraph['GT'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] == 1:
        b1.append(ei1[i])
        b2.append(ei2[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    edge['color'] = ec1
g.show('data/Vis/BaselineGraphGT.html')

In [ ]:
basebetaall = pd.read_csv('data/xAI Comparison/heteroBetaExpxAIProbsVSGT.csv')
basebetaidx = basebetaall[basebetaall['Idx'] == 13213]
basebetagraph = basebetaidx[basebetaidx['Run'] == 6]
p_edges = basebetagraph[basebetagraph['GT'] == 1]
neg_edges = basebetagraph[basebetagraph['GT'] == 1]
fn_edges = neg_edges[neg_edges['Beta Probability'] < 0.5]
p_set = set()
p1s = list(p_edges['P1'])
p2s = list(p_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    p_set.add((p1, p2))
fn_set = set()
width = dict()
wtr = set()
p1s = list(fn_edges['P1'])
p2s = list(fn_edges['P2'])
prob = list(fn_edges['Beta Probability'])
pr = []
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    fn_set.add((p1, p2))
    if (p1, p2) not in wtr:
        if p1 in width.keys():
            width[p1][p2] = prob[i]
        else:
            width[p1] = {p2: prob[i]}
g = Network(notebook=True, directed=True)
ei1 = list(basebetagraph['P1'])
ei2 = list(basebetagraph['P2'])
prob = list(basebetagraph['Beta Probability'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
if len(fn_set) > 0:
    for (p1, p2) in fn_set:
        b1.append(p1)
        b2.append(p2)
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in p_set:
        if (p1, p2) in fn_set:
            edge['color'] = ec0_1
        else:
            edge["color"] = ec1
    else:
        edge["color"] = ec0_0
g.show('data/Vis/HeterophilicGraphBeta.html')

In [ ]:
basegnnall = pd.read_csv('data/xAI Comparison/heteroGNNExpxAIProbsVSGT.csv')
basegnnidx = basegnnall[basegnnall['Idx'] == 13213]
br = 0
bestacc = 0
bestf1 = 0
runs = np.unique(basegnnidx['Run'])
for run in runs:
    curr = basegnnidx[basegnnidx['Run'] == run]
    probs = torch.tensor(list(curr['GNN Probability']), dtype=torch.float32)
    gt = torch.tensor(list(curr['GT']), dtype=torch.int64)
    met = groundtruth_metrics(probs, gt)
    acc = met[0]
    f1 = met[3]
    if acc >= bestacc and f1 >= bestf1:
        br = run
        bestacc = acc
        bestf1 = f1
print(bestacc, bestf1, br)

In [ ]:
basegnngraph = basegnnidx[basegnnidx['Run'] == br]
p_edges = basegnngraph[basegnngraph['GT'] == 1]
neg_edges = basegnngraph[basegnngraph['GT'] == 1]
fn_edges = neg_edges[neg_edges['GNN Probability'] < 0.5]
p_set = set()
p1s = list(p_edges['P1'])
p2s = list(p_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    p_set.add((p1, p2))
fn_set = set()
p1s = list(fn_edges['P1'])
p2s = list(fn_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    fn_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(basegnngraph['P1'])
ei2 = list(basegnngraph['P2'])
prob = list(basegnngraph['GNN Probability'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
if len(fn_set) > 0:
    for (p1, p2) in fn_set:
        b1.append(p1)
        b2.append(p2)
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in p_set:
        if (p1, p2) in fn_set:
            edge['color'] = ec0_1
        else:
            edge["color"] = ec1
    else:
        edge["color"] = ec0_0
g.show('data/Vis/HeterophilicGraphGNN.html')

In [ ]:
basegnnall = pd.read_csv('data/xAI Comparison/heteroGNNExpxAIProbsVSGT.csv')
basegnnidx = basegnnall[basegnnall['Idx'] == 13213]
basegnngraph = basegnnidx[basegnnidx['Run'] == br]
g = Network(notebook=True, directed=True)
ei1 = list(basegnngraph['P1'])
ei2 = list(basegnngraph['P2'])
prob = list(basegnngraph['GT'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] == 1:
        b1.append(ei1[i])
        b2.append(ei2[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    edge['color'] = ec1
g.show('data/Vis/heterophilicGraphGT.html')

In [ ]:
basebetaall = pd.read_csv('data/xAI Comparison/lessinformBetaExpxAIProbsVSGT.csv')
basebetaidx = basebetaall[basebetaall['Idx'] == 11890]
br = 0
bestacc = 0
bestf1 = 0
runs = np.unique(basebetaidx['Run'])
for run in runs:
    curr = basebetaidx[basebetaidx['Run'] == run]
    probs = torch.tensor(list(curr['Beta Probability']), dtype=torch.float32)
    gt = torch.tensor(list(curr['GT']), dtype=torch.int64)
    met = groundtruth_metrics(probs, gt)
    acc = met[0]
    f1 = met[3]
    if acc >= bestacc and f1 >= bestf1:
        br = run
        bestacc = acc
        bestf1 = f1
print(bestacc, bestf1, br)

In [ ]:
basebetagraph = basebetaidx[basebetaidx['Run'] == br]
p_edges = basebetagraph[basebetagraph['GT'] == 1]
neg_edges = basebetagraph[basebetagraph['GT'] == 1]
fn_edges = neg_edges[neg_edges['Beta Probability'] < 0.5]
p_set = set()
p1s = list(p_edges['P1'])
p2s = list(p_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    p_set.add((p1, p2))
fn_set = set()
width = dict()
wtr = set()
p1s = list(fn_edges['P1'])
p2s = list(fn_edges['P2'])
prob = list(fn_edges['Beta Probability'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    fn_set.add((p1, p2))
    if (p1, p2) not in wtr:
        if p1 in width.keys():
            width[p1][p2] = prob[i]
        else:
            width[p1] = {p2: prob[i]}
g = Network(notebook=True, directed=True)
ei1 = list(basebetagraph['P1'])
ei2 = list(basebetagraph['P2'])
prob = list(basebetagraph['Beta Probability'])
b1 = []
b2 = []
n = []
c = []
width = dict()
wtr = set()
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) and (ei2[i], ei1[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            if ei2[i] in width.keys():
                width[ei2[i]][ei1[i]] = prob[i]
            else:
                width[ei2[i]] = {ei1[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
            wtr.add((ei2[i], ei1[i]))
if len(fn_set) > 0:
    for (p1, p2) in fn_set:
        b1.append(p1)
        b2.append(p2)
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5 * width[b1[i]][b2[i]]
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in p_set:
        if (p1, p2) in fn_set:
            edge['color'] = lighten(ec0_1, width[p1][p2])
        else:
            edge["color"] = lighten(ec1, width[p1][p2])
    else:
        edge["color"] = lighten(ec0_0, width[p1][p2])
g.show('data/Vis/LessInformGraphBeta.html')

In [ ]:
basegnnall = pd.read_csv('data/xAI Comparison/lessinformGNNExpxAIProbsVSGT.csv')
basegnnidx = basegnnall[basegnnall['Idx'] == 11890]
basegnngraph = basegnnidx[basegnnidx['Run'] == 24]
p_edges = basegnngraph[basegnngraph['GT'] == 1]
neg_edges = basegnngraph[basegnngraph['GT'] == 1]
fn_edges = neg_edges[neg_edges['GNN Probability'] < 0.5]
p_set = set()
p1s = list(p_edges['P1'])
p2s = list(p_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    p_set.add((p1, p2))
fn_set = set()
p1s = list(fn_edges['P1'])
p2s = list(fn_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    fn_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(basegnngraph['P1'])
ei2 = list(basegnngraph['P2'])
prob = list(basegnngraph['GNN Probability'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
if len(fn_set) > 0:
    for (p1, p2) in fn_set:
        b1.append(p1)
        b2.append(p2)
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in p_set:
        if (p1, p2) in fn_set:
            edge['color'] = ec0_1
        else:
            edge["color"] = ec1
    else:
        edge["color"] = ec0_0
g.show('data/Vis/LessInformGraphGNN.html')

In [ ]:
basegnnall = pd.read_csv('data/xAI Comparison/lessinformGNNExpxAIProbsVSGT.csv')
basegnnidx = basegnnall[basegnnall['Idx'] == 11890]
basegnngraph = basegnnidx[basegnnidx['Run'] == 24]
g = Network(notebook=True, directed=True)
ei1 = list(basegnngraph['P1'])
ei2 = list(basegnngraph['P2'])
prob = list(basegnngraph['GT'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] == 1:
        b1.append(ei1[i])
        b2.append(ei2[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    edge['color'] = ec1
g.show('data/Vis/LessInformGraphGT.html')

In [ ]:
basebetaall = pd.read_csv('data/xAI Comparison/moreinformBetaExpxAIProbsVSGT.csv')
basebetaidx = basebetaall[basebetaall['Idx'] == 12651]
br = 0
bestacc = 0
bestf1 = 0
runs = np.unique(basebetaidx['Run'])
for run in runs:
    curr = basebetaidx[basebetaidx['Run'] == run]
    probs = torch.tensor(list(curr['Beta Probability']), dtype=torch.float32)
    gt = torch.tensor(list(curr['GT']), dtype=torch.int64)
    met = groundtruth_metrics(probs, gt)
    acc = met[0]
    f1 = met[3]
    if acc >= bestacc and f1 >= bestf1:
        br = run
        bestacc = acc
        bestf1 = f1
print(bestacc, bestf1, br)

In [ ]:
basebetagraph = basebetaidx[basebetaidx['Run'] == br]
p_edges = basebetagraph[basebetagraph['GT'] == 1]
neg_edges = basebetagraph[basebetagraph['GT'] == 1]
fn_edges = neg_edges[neg_edges['Beta Probability'] < 0.5]
p_set = set()
p1s = list(p_edges['P1'])
p2s = list(p_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    p_set.add((p1, p2))
fn_set = set()
width = dict()
wtr = set()
p1s = list(fn_edges['P1'])
p2s = list(fn_edges['P2'])
prob = list(fn_edges['Beta Probability'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    fn_set.add((p1, p2))
    if (p1, p2) not in wtr:
        if p1 in width.keys():
            width[p1][p2] = prob[i]
        else:
            width[p1] = {p2: prob[i]}
g = Network(notebook=True, directed=True)
ei1 = list(basebetagraph['P1'])
ei2 = list(basebetagraph['P2'])
prob = list(basebetagraph['Beta Probability'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
if len(fn_set) > 0:
    for (p1, p2) in fn_set:
        b1.append(p1)
        b2.append(p2)
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5 * width[b1[i]][b2[i]]
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in p_set:
        if (p1, p2) in fn_set:
            edge['color'] = lighten(ec0_1, width[p1][p2])
        else:
            edge["color"] = lighten(ec1, width[p1][p2])
    else:
        edge["color"] = lighten(ec0_0, width[p1][p2])
g.show('data/Vis/MoreInformGraphBeta.html')

In [ ]:
basegnnall = pd.read_csv('data/xAI Comparison/moreinformGNNExpxAIProbsVSGT.csv')
basegnnidx = basegnnall[basegnnall['Idx'] == 12651]
basegnngraph = basegnnidx[basegnnidx['Run'] == 14]
p_edges = basegnngraph[basegnngraph['GT'] == 1]
neg_edges = basegnngraph[basegnngraph['GT'] == 1]
fn_edges = neg_edges[neg_edges['GNN Probability'] < 0.5]
p_set = set()
p1s = list(p_edges['P1'])
p2s = list(p_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    p_set.add((p1, p2))
fn_set = set()
p1s = list(fn_edges['P1'])
p2s = list(fn_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    fn_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(basegnngraph['P1'])
ei2 = list(basegnngraph['P2'])
prob = list(basegnngraph['GNN Probability'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
if len(fn_set) > 0:
    for (p1, p2) in fn_set:
        b1.append(p1)
        b2.append(p2)
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in p_set:
        if (p1, p2) in fn_set:
            edge['color'] = ec0_1
        else:
            edge["color"] = ec1
    else:
        edge["color"] = ec0_0
g.show('data/Vis/MoreInformGraphGNN.html')

In [ ]:
basegnnall = pd.read_csv('data/xAI Comparison/moreinformGNNExpxAIProbsVSGT.csv')
basegnnidx = basegnnall[basegnnall['Idx'] == 12651]
basegnngraph = basegnnidx[basegnnidx['Run'] == 14]
g = Network(notebook=True, directed=True)
ei1 = list(basegnngraph['P1'])
ei2 = list(basegnngraph['P2'])
prob = list(basegnngraph['GT'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] == 1:
        b1.append(ei1[i])
        b2.append(ei2[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    edge['color'] = ec1
g.show('data/Vis/MoreInformGraphGT.html')

In [ ]:
basebetaall = pd.read_csv('data/xAI Comparison/unfairBetaExpxAIProbsVSGT.csv')
basebetaidx = basebetaall[basebetaall['Idx'] == 37]
br = 0
bestacc = 0
bestf1 = 0
runs = np.unique(basebetaidx['Run'])
for run in runs:
    curr = basebetaidx[basebetaidx['Run'] == run]
    probs = torch.tensor(list(curr['Beta Probability']), dtype=torch.float32)
    gt = torch.tensor(list(curr['GT']), dtype=torch.int64)
    met = groundtruth_metrics(probs, gt)
    acc = met[0]
    f1 = met[3]
    if acc >= bestacc and f1 >= bestf1:
        br = run
        bestacc = acc
        bestf1 = f1
print(bestacc, bestf1, br)

In [ ]:
basebetagraph = basebetaidx[basebetaidx['Run'] == br]
p_edges = basebetagraph[basebetagraph['GT'] == 1]
neg_edges = basebetagraph[basebetagraph['GT'] == 1]
fn_edges = neg_edges[neg_edges['Beta Probability'] < 0.5]
p_set = set()
p1s = list(p_edges['P1'])
p2s = list(p_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    p_set.add((p1, p2))
fn_set = set()
width = dict()
wtr = set()
p1s = list(fn_edges['P1'])
p2s = list(fn_edges['P2'])
prob = list(fn_edges['Beta Probability'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    fn_set.add((p1, p2))
    if (p1, p2) not in wtr:
        if p1 in width.keys():
            width[p1][p2] = prob[i]
        else:
            width[p1] = {p2: prob[i]}
g = Network(notebook=True, directed=True)
ei1 = list(basebetagraph['P1'])
ei2 = list(basebetagraph['P2'])
prob = list(basebetagraph['Beta Probability'])
b1 = []
b2 = []
n = []
c = []
width = dict()
wtr = set()
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
        if (ei1[i], ei2[i]) not in wtr:
            if ei1[i] in width.keys():
                width[ei1[i]][ei2[i]] = prob[i]
            else:
                width[ei1[i]] = {ei2[i]: prob[i]}
            wtr.add((ei1[i], ei2[i]))
if len(fn_set) > 0:
    for (p1, p2) in fn_set:
        b1.append(p1)
        b2.append(p2)
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    w = 5 * width[b1[i]][b2[i]]
    g.add_edge(b1[i], b2[i], width=w)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in p_set:
        if (p1, p2) in fn_set:
            edge['color'] = lighten(ec0_1, width[p1][p2])
        else:
            edge["color"] = lighten(ec1, width[p1][p2])
    else:
        edge["color"] = lighten(ec0_0, width[p1][p2])
g.show('data/Vis/UnfairGraphBeta.html')

In [ ]:
basegnnall = pd.read_csv('data/xAI Comparison/unfairGNNExpxAIProbsVSGT.csv')
basegnnidx = basegnnall[basegnnall['Idx'] == 37]
basegnngraph = basegnnidx[basegnnidx['Run'] == 13]
p_edges = basegnngraph[basegnngraph['GT'] == 1]
neg_edges = basegnngraph[basegnngraph['GT'] == 1]
fn_edges = neg_edges[neg_edges['GNN Probability'] < 0.5]
p_set = set()
p1s = list(p_edges['P1'])
p2s = list(p_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    p_set.add((p1, p2))
fn_set = set()
p1s = list(fn_edges['P1'])
p2s = list(fn_edges['P2'])
for i in range(len(p1s)):
    p1 = p1s[i]
    p2 = p2s[i]
    fn_set.add((p1, p2))
g = Network(notebook=True, directed=True)
ei1 = list(basegnngraph['P1'])
ei2 = list(basegnngraph['P2'])
prob = list(basegnngraph['GNN Probability'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] >= 0.5:
        b1.append(ei1[i])
        b2.append(ei2[i])
if len(fn_set) > 0:
    for (p1, p2) in fn_set:
        b1.append(p1)
        b2.append(p2)
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    p1 = edge['from']
    p2 = edge['to']
    if (p1, p2) in p_set:
        if (p1, p2) in fn_set:
            edge['color'] = ec0_1
        else:
            edge["color"] = ec1
    else:
        edge["color"] = ec0_0
g.show('data/Vis/UnfairGraphGNN.html')

In [ ]:
basegnnall = pd.read_csv('data/xAI Comparison/unfairGNNExpxAIProbsVSGT.csv')
basegnnidx = basegnnall[basegnnall['Idx'] == 37]
basegnngraph = basegnnidx[basegnnidx['Run'] == 13]
g = Network(notebook=True, directed=True)
ei1 = list(basegnngraph['P1'])
ei2 = list(basegnngraph['P2'])
prob = list(basegnngraph['GT'])
b1 = []
b2 = []
n = []
c = []
for i in range(len(ei1)):
    if prob[i] == 1:
        b1.append(ei1[i])
        b2.append(ei2[i])
u1 = np.unique(b1)
u2 = np.unique(b2)
for i in u1:
    if i not in n:
        n.append(i)
        c.append('Black')
for i in u2:
    if i not in n:
        n.append(i)
        c.append('Black')
g.add_nodes(n, color=c)
for i in range(0, len(b1)):
    g.add_edge(b1[i], b2[i], width=5)
for edge in g.get_edges():
    edge['color'] = ec1
g.show('data/Vis/UnfairGraphGT.html')

In [ ]:
df = pd.read_csv('data/xAI/baseBetaExpxAIRunBestProbsVSGT.csv')
df['Category'] = df['GT'].apply(lambda x: 'False Positive' if x == 0 else 'True Positive')
df = df.sort_values(by=['Category'])
p = sns.ecdfplot(data=df, x="Probability", hue="Category", palette=['grey', 'green']).set_title('SG-BASE eCDF')

In [ ]:
df = pd.read_csv('data/xAI/heteroBetaExpxAIRunBestProbsVSGT.csv')
df['Category'] = df['GT'].apply(lambda x: 'False Positive' if x == 0 else 'True Positive')
df = df.sort_values(by=['Category'])
p = sns.ecdfplot(data=df, x="Probability", hue="Category", palette=['grey', 'green']).set_title('SG-HETEROPHILIC eCDF')

In [ ]:
df = pd.read_csv('data/xAI/unfairBetaExpxAIRunBestProbsVSGT.csv')
df['Category'] = df['GT'].apply(lambda x: 'False Positive' if x == 0 else 'True Positive')
df = df.sort_values(by=['Category'])
p = sns.ecdfplot(data=df, x="Probability", hue="Category", palette=['grey', 'green']).set_title('SG-UNFAIR eCDF')

In [ ]:
df = pd.read_csv('data/xAI/lessinformBetaExpxAIRunBestProbsVSGT.csv')
df['Category'] = df['GT'].apply(lambda x: 'False Positive' if x == 0 else 'True Positive')
df = df.sort_values(by=['Category'])
p = sns.ecdfplot(data=df, x="Probability", hue="Category", palette=['grey', 'green']).set_title('SG-LESSINFORM eCDF')

In [ ]:
df = pd.read_csv('data/xAI/moreinformBetaExpxAIRunBestProbsVSGT.csv')
df['Category'] = df['GT'].apply(lambda x: 'False Positive' if x == 0 else 'True Positive')
df = df.sort_values(by=['Category'])
p = sns.ecdfplot(data=df, x="Probability", hue="Category", palette=['grey', 'green']).set_title('SG-MOREINFORM eCDF')